In [2]:
import json
import csv
import tweepy
import re
import logging
import time


In [3]:
"""
INPUTS:
    consumer_key, consumer_secret, access_token, access_token_secret: codes 
    telling twitter that we are authorized to access this data
    hashtag_phrase: the combination of hashtags to search for
OUTPUTS:
    none, simply save the tweet info to a spreadsheet
"""

logging.basicConfig(filename='log_filename.txt', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logging.debug('This is a log message.')

ckey = "x0Ju7gGuk7KaSlVbhsjs0f1qb"
csecret = "mB5UTVdJPnCD5xkzV5FHtAxQCUAeLwv7Yv832R6Rnul4F4lgL4"
atoken = "2659405784-tmtOps0vaCq4ut9eSSA6Im9TmsTXs3xABFGuHTX"
asecret = "wvlWKxCh6tj2qX66sLQnGQVobWQKnM7iI2MUaRi2z0ShK"


def search_for_hashtags(consumer_key, 
                        consumer_secret, 
                        access_token, 
                        access_token_secret, 
                        hashtag_phrase):
    
    #create authentication for accessing Twitter
#    auth = tweepy.OAuthHandler('consumer_key':ckey, 'consumer_secret':csecret)
#    auth.set_access_token('access_token_key':atoken, 'access_token_secret':asecret)

    OAUTH_KEYS = {'consumer_key':ckey, 
                  'consumer_secret':csecret,
                  'access_token_key':atoken, 
                  'access_token_secret':asecret}
    
    auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])

    #initialize Tweepy API
    api = tweepy.API(auth, wait_on_rate_limit=True)
        
    #get the name of the spreadsheet we will write to
    fname = '_'.join(re.findall(r"#(\w+)", 'RawData'))
    

    #open the spreadsheet we will write to
    with open('RawData%s.csv'%(fname), 'w', encoding="utf-8") as file:

        w = csv.writer(file)

        #write header row to spreadsheet
        w.writerow(['Timestamp', 
                    'Tweet Text',
                    'Hive Owners',
                    'Likes', 
                    'Retweets',
                    'Followers',
                    'Hashtags', 
                    'Tweets ID',
                    'Likes & RTs'
                    ])
        
        
        #for each tweet matching our hashtags, write relevant info to the spreadsheet
        c = tweepy.Cursor(api.search, q=hashtag_phrase+' ', \
                        tweet_mode='extended').items()

        first_date = ""
        last_date = ""
        while True:
            try:
                tweet = c.next()
                last_date = tweet.created_at
                if (first_date == ""):
                    first_date = tweet.created_at
                w.writerow([tweet.created_at,
                    tweet.full_text.replace('\n',' ').encode('utf-8'),
                    tweet.user.screen_name.encode('utf-8'),
                    tweet.favorite_count,
                    tweet.retweet_count,
                    tweet.user.followers_count,
                    [e['text'] for e in tweet._json['entities']['hashtags']],
                    tweet.id
                   ])
            except tweepy.TweepError:
                print('tweepy error')
                time.sleep(60 * 15)
                continue
            except StopIteration:
                print('stop iteration')
                break

        w.writerow(['From Date', last_date])
        w.writerow(['To Date', first_date])
        

In [ ]:
consumer_key = "x0Ju7gGuk7KaSlVbhsjs0f1qb"
consumer_secret = "mB5UTVdJPnCD5xkzV5FHtAxQCUAeLwv7Yv832R6Rnul4F4lgL4"
access_token = "2659405784-tmtOps0vaCq4ut9eSSA6Im9TmsTXs3xABFGuHTX"
access_token_secret = "wvlWKxCh6tj2qX66sLQnGQVobWQKnM7iI2MUaRi2z0ShK"
    
hashtag_phrase = '"#hive" OR "#hivetwitter" OR "#hivechat" OR "#hiveisalive" OR "#hivelift" OR "#hivetask" OR "#hivepud" OR "#leofinance" OR "#Hivecomments" OR "#joinhive" OR "#3speak" OR "#threespeak" OR "#hive5" OR "leofinance.io" OR "#splinterlands" OR "$hive" OR "hive.blog" OR "peakd.com" OR "ecency.com" OR "hive.io" OR "hiveonboard" OR "hivetips"'



if __name__ == '__main__':
    try:
        search_for_hashtags(consumer_key, consumer_secret, access_token, access_token_secret, hashtag_phrase)
    except Exception as e:
        print(e)